In [1]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re



def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting testutility.py


# Write Yaml file

In [3]:
%%writefile file.yaml
file_type: csv
dataset_name: testfile
file_name: hospitals
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - cms_certification_num
    - name
    - address
    - city 
    - state
    - zip5
    - beds
    - phone_number
    - homepage_url
    - chargemaster_url
    - last_edited_by_username

Overwriting file.yaml


In [11]:
# Read config file
import testutility as util
config_data = util.read_config_file("file.yaml")

In [12]:
config_data['inbound_delimiter']

','

In [13]:
#inspecting data of config file
config_data

{'file_type': 'csv',
 'dataset_name': 'testfile',
 'file_name': 'hospitals',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['cms_certification_num',
  'name',
  'address',
  'city',
  'state',
  'zip5',
  'beds',
  'phone_number',
  'homepage_url',
  'chargemaster_url',
  'last_edited_by_username']}

In [9]:
import pandas as pd
df_hospital = pd.read_csv("archive/hospitals.csv")
df_hospital.head()

,cms_certification_num,name,address,city,state,zip5,beds,phone_number,homepage_url,chargemaster_url,last_edited_by_username
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,420,3347938701,https://www.southeasthealth.org/southeast-heal...,https://www.southeasthealth.org/financial-info...,captainstabs
1,010005,MARSHALL MEDICAL CENTERS SOUTH CAMPUS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,240,2565938310,https://www.marshallmedical.org/,https://www.marshallmedical.org/patients-visit...,captainstabs
2,010006,NORTH ALABAMA MEDICAL CENTER,1701 VETERANS DRIVE,FLORENCE,AL,35630,338,2567688400,NaN,NaN,NaN
3,010007,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,99,3344933541,NaN,NaN,NaN
4,010008,CRENSHAW COMMUNITY HOSPITAL,101 HOSPITAL CIRCLE,LUVERNE,AL,36049,65,3343353374,NaN,NaN,NaN


In [14]:
print("columns of files are:" ,  df_hospital.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['cms_certification_num', 'name', 'address', 'city', 'state', 'zip5',
       'beds', 'phone_number', 'homepage_url', 'chargemaster_url',
       'last_edited_by_username'],
      dtype='object')
columns of YAML are: ['cms_certification_num', 'name', 'address', 'city', 'state', 'zip5', 'beds', 'phone_number', 'homepage_url', 'chargemaster_url', 'last_edited_by_username']


In [15]:
df_hospital.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5992 entries, 0 to 5991
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   cms_certification_num    5992 non-null   object
 1   name                     5992 non-null   object
 2   address                  5992 non-null   object
 3   city                     5992 non-null   object
 4   state                    5992 non-null   object
 5   zip5                     5992 non-null   int64 
 6   beds                     5992 non-null   int64 
 7   phone_number             5992 non-null   int64 
 8   homepage_url             1799 non-null   object
 9   chargemaster_url         1799 non-null   object
 10  last_edited_by_username  1799 non-null   object
dtypes: int64(3), object(8)
memory usage: 515.1+ KB


In [16]:
df_hospital.describe()

,zip5,beds,phone_number
count,5992.000000,5992.000000,5.992000e+03
mean,53785.892857,162.129172,5.859922e+09
std,26990.331200,210.167654,2.362392e+09
min,603.000000,2.000000,9.369338e+08
25%,32610.000000,26.000000,3.613390e+09
50%,55104.500000,80.000000,6.054065e+09
75%,76125.250000,214.000000,8.015615e+09
max,99929.000000,2891.000000,9.898943e+09
